<a href="https://colab.research.google.com/github/Shreyas1308/gunshot-classification/blob/main/Build_and_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import panda as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models, callbacks
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

Loading the dataset

In [ ]:
# CSV should have columns: [filename, gun_type]
labels_df = pd.read_csv('')
img_dir = ""

In [ ]:
X = []
y = []

for idx, row in labels_df.iterrows():
    img_path = os.path.join(img_dir, row['filename'])
    img = Image.open(img_path).convert('RGB')
    img = img.resize((128,128))  # Just in case
    X.append(np.array(img)/255.0)  # Normalize to [0,1]
    y.append(row['gun_type'])

X = np.array(X)
gun_types = sorted(list(set(y)))
type2idx = {gt: i for i, gt in enumerate(gun_types)}
y = np.array([type2idx[gt] for gt in y])

print("Shape of X:", X.shape)
print("Shape of y:", y.shape)
print("Unique classes:", gun_types)

Split Data

In [ ]:
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

num_classes = len(gun_types)
y_train_cat = to_categorical(y_train, num_classes)
y_val_cat = to_categorical(y_val, num_classes)
y_test_cat = to_categorical(y_test, num_classes)

Visualize samples

In [ ]:
plt.figure(figsize=(10,5))
for i in range(5):
    plt.subplot(1,5,i+1)
    plt.imshow(X_train[i])
    plt.title(f"Class: {gun_types[y_train[i]]}")
    plt.axis('off')
plt.show()

Building the training Model

In [ ]:
input_shape = X_train.shape[1:]

model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=input_shape),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),
    layers.Flatten(),
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.summary()

Callbacks

In [ ]:
cb_earlystop = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
cb_modelckpt = callbacks.ModelCheckpoint('best_gun_type_model.h5', save_best_only=True)

Start Training

In [ ]:
history = model.fit(
    X_train, y_train_cat,
    epochs=30,
    batch_size=32,
    validation_data=(X_val, y_val_cat),
    callbacks=[cb_earlystop, cb_modelckpt]
)

Evaluation

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test_cat)
print(f"Test accuracy: {test_acc:.2%}")

Confusion Matrix

In [ ]:
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
cm = confusion_matrix(y_test, y_pred_classes)
disp = ConfusionMatrixDisplay(cm, display_labels=gun_types)
disp.plot(xticks_rotation='vertical')
plt.show()

Saving the model

In [ ]:
#model.save('gun_type_model_final.h5')